<a href="https://colab.research.google.com/github/kakakafafafa888/Adipose_Ghost_Analysis/blob/main/00_Data_Acquisition_and_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🛠️ 一键构建项目基地 (Run This Cell)
import os
from google.colab import drive

# 1. 挂载 Google Drive (把云盘插到 Colab 这台电脑上)
print("正在挂载 Google Drive...")
drive.mount('/content/drive')

# 2. 定义你的项目主目录路径
# 注意：不要改这个路径，保持规范
project_root = '/content/drive/MyDrive/Project_Adipose_Ghost'

# 3. 定义内部结构 (根据项目书要求)
structure = [
    "00_RawData",       # 存放 GEO/Atlas 原始数据
    "01_Processing",    # 存放处理后的 h5ad 中间文件
    "02_Figures",       # 存放 PDF/SVG 矢量图
    "03_Scripts",       # 存放你的 .ipynb 脚本备份
    "resources"         # 存放基因列表等辅助文件
]

# 4. 循环创建目录
print(f"\n正在构建目录结构于: {project_root}")

if not os.path.exists(project_root):
    os.makedirs(project_root)
    print(f"✅ 创建主目录: {project_root}")
else:
    print(f"ℹ️ 主目录已存在: {project_root}")

for folder in structure:
    folder_path = os.path.join(project_root, folder)
    # exist_ok=True 表示如果文件夹已存在，不会报错，防止你重复运行炸掉
    os.makedirs(folder_path, exist_ok=True)
    print(f"   └─ 📂 检查/创建子目录: {folder}")

# 5. 创建一个 README 文件作为初始化标志
readme_path = os.path.join(project_root, "README.txt")
with open(readme_path, "w") as f:
    f.write("Project: Adipose Ghost Phase Mechanism Analysis\n")
    f.write("Created by: Your Name (Student)\n")
    f.write("Supervised by: Tiancai (Professor)\n")
    f.write("Goal: Validate Ischemic Preconditioning Hypothesis ")

print("\n🎉 基地建设完毕。你可以去 Google Drive 查看了。")

正在挂载 Google Drive...


ValueError: mount failed

In [ ]:
# @title ⬇️ 云端极速下载器 (GEO Download Protocol)
import os
from google.colab import drive

# ================= 配置区域 =================
# 1. 粘贴你刚才复制的链接到这里
DOWNLOAD_URL = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE189783&format=file"

# 2. 设置保存的文件名 (建议保持原名)
OUTPUT_FILENAME = "GSE189783_RAW.tar"
# ===========================================

# 1. 挂载云盘 (如果已经挂载过，系统会自动跳过)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 切换到由于 "Source 17" 指定的数据仓库
# 路径必须严格对应我们刚才创建的结构
target_dir = '/content/drive/MyDrive/Project_Adipose_Ghost/00_RawData'

if os.path.exists(target_dir):
    os.chdir(target_dir)
    print(f"📂 已锁定目标仓库: {target_dir}")
else:
    print(f"❌ 错误：找不到目录 {target_dir}。请先运行之前的'基地建设'脚本！")
    # 这里加个终止，防止下错地方
    raise SystemExit("目录不存在，请检查路径。")

# 3. 执行下载 (使用 Linux wget 工具)
# -c: 断点续传 (网络断了也不怕，重新运行会接着下)
# -O: 指定输出文件名
print(f"🚀 开始从 NCBI 抽取数据: {OUTPUT_FILENAME} ...")
cmd = f'wget -c "{DOWNLOAD_URL}" -O "{OUTPUT_FILENAME}"'
os.system(cmd)

# 4. 检查文件大小，确认下载成功
file_size = os.path.getsize(OUTPUT_FILENAME) / (1024 * 1024) # 转为 MB
print(f"✅ 下载完成！文件大小: {file_size:.2f} MB")

# 5. (可选) 如果是 .tar 文件，自动解压
if OUTPUT_FILENAME.endswith('.tar'):
    print("📦 检测到归档文件，正在自动解压...")
    # 创建一个子文件夹放解压后的文件，避免混乱
    extract_folder = OUTPUT_FILENAME.replace('.tar', '_unzipped')
    os.makedirs(extract_folder, exist_ok=True)
    # 解压命令
    os.system(f'tar -xvf "{OUTPUT_FILENAME}" -C "{extract_folder}"')
    print(f"🎉 解压完毕！数据位于: {target_dir}/{extract_folder}")

📂 已锁定目标仓库: /content/drive/MyDrive/Project_Adipose_Ghost/00_RawData
🚀 开始从 NCBI 抽取数据: GSE189783_RAW.tar ...
✅ 下载完成！文件大小: 1127.06 MB
📦 检测到归档文件，正在自动解压...
🎉 解压完毕！数据位于: /content/drive/MyDrive/Project_Adipose_Ghost/00_RawData/GSE189783_RAW_unzipped


In [ ]:
# @title 🗑️ 云端焚化炉：精准清除垃圾数据
import os
import shutil
from google.colab import drive

# 1. 挂载云盘 (常规操作)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 定义垃圾场的坐标
# 我们的目标在 00_RawData 下
base_path = '/content/drive/MyDrive/Project_Adipose_Ghost/00_RawData'

# 3. 定义要删除的目标清单 (Target List)
# 包含：那个巨大的 .tar 包，以及解压出来的文件夹
targets_to_delete = [
    "GSE189783_RAW.tar",            # 原始压缩包
    "GSE189783_RAW_unzipped",       # 如果你解压到了这个文件夹
    "GSE189783"                     # 以防万一你用了不同的文件夹名
]

print(f"🔥 启动焚化程序，目标区域: {base_path}")

# 4. 执行清除任务
for item in targets_to_delete:
    full_path = os.path.join(base_path, item)

    if os.path.exists(full_path):
        try:
            if os.path.isfile(full_path):
                os.remove(full_path) # 删除文件
                print(f"   ❌ 已粉碎文件: {item}")
            elif os.path.isdir(full_path):
                shutil.rmtree(full_path) # 递归删除文件夹及其内容
                print(f"   💥 已铲除目录: {item}")
        except Exception as e:
            print(f"   ⚠️ 删除失败 {item}: {e}")
    else:
        print(f"   💨 目标不存在 (可能已经删过了): {item}")

# 5. 再次列出目录内容，确认清理干净
print("\n🧹 现在的 00_RawData 目录内容:")
current_files = os.listdir(base_path)
if not current_files:
    print("   (空空如也)")
else:
    for f in current_files:
        print(f"   - {f}")

print("\n✅ 清除完毕。你的工作区现在干净了。")

🔥 启动焚化程序，目标区域: /content/drive/MyDrive/Project_Adipose_Ghost/00_RawData
   ❌ 已粉碎文件: GSE189783_RAW.tar
   💥 已铲除目录: GSE189783_RAW_unzipped
   💨 目标不存在 (可能已经删过了): GSE189783

🧹 现在的 00_RawData 目录内容:
   (空空如也)

✅ 清除完毕。你的工作区现在干净了。


In [ ]:
# @title ⬇️ GSE138826 双通道下载
import os
from google.colab import drive

# 切换目录
os.chdir('/content/drive/MyDrive/Project_Adipose_Ghost/00_RawData')

# 创建专属文件夹
if not os.path.exists('GSE138826'):
    os.makedirs('GSE138826')
os.chdir('GSE138826')

# 1. 下载 TXT (优先尝试)
print("🚀 正在下载 Expression Matrix (TXT)...")
!wget -c "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE138826&format=file&file=GSE138826%5Fexpression%5Fmatrix%2Etxt%2Egz" -O GSE138826_expression_matrix.txt.gz

# 2. 下载 RDS (保底备份)
print("🚀 正在下载 RDS Data (Backup)...")
!wget -c "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE138826&format=file&file=GSE138826%5Fregen%5Fdata%2Erds%2Egz" -O GSE138826_regen_data.rds.gz

print("✅ 下载完成。")

🚀 正在下载 Expression Matrix (TXT)...
--2025-11-24 04:33:20--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE138826&format=file&file=GSE138826%5Fexpression%5Fmatrix%2Etxt%2Egz
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155522488 (148M) [application/octet-stream]
Saving to: ‘index.html?acc=GSE138826&format=file&file=GSE138826_expression_matrix.txt.gz’

index.html?acc=GSE1 100%[===================>] 148.32M  27.4MB/s    in 4.7s    

2025-11-24 04:33:25 (31.6 MB/s) - ‘index.html?acc=GSE138826&format=file&file=GSE138826_expression_matrix.txt.gz’ saved [155522488/155522488]

🚀 正在下载 RDS Data (Backup)...
--2025-11-24 04:33:25--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE138826&format=file&file=GSE138826%5Fregen%5Fdata%2Erds%2Egz
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.

In [7]:
# @title 🧐 验尸报告：检查 TXT 是否携带时间信息
import pandas as pd
!pip install scanpy
import scanpy as sc

# 1. 尝试直接读取前几行 (Peek)
# 不需要解压 gz，pandas 会自动处理
file_path = '/content/drive/MyDrive/Project_Adipose_Ghost/00_RawData/GSE138826_Muscle_CTX_TimeCourse/index.html?acc=GSE138826&format=file&file=GSE138826_expression_matrix.txt.gz'


print("正在读取矩阵表头...")
# 只读第一行，看看列名长什么样
header = pd.read_csv(file_path, sep='\t', nrows=1)
print(f"列名示例 (前5个): {list(header.columns[:5])}")

# 2. 智能判断
first_col = header.columns[0]
if "Day" in first_col or "0.5" in first_col or "_" in first_col:
    print("\n🎉 大吉大利！列名里看起来包含时间信息！")
    print("   -> 我们可以直接用 Python 处理，不需要 R 语言桥接了！")
    print("   -> 下一步：直接读取整个矩阵并提取 Timepoint。")

    # 如果确认是大吉，直接加载整个数据 (内存够的话)
    # adata = sc.read_text(file_path).T # 通常txt是 gene x cell，scanpy需要转置

else:
    print("\n⚠️ 警报：列名看起来只是纯 Barcode (如 AAACCTG...).")
    print("   -> TXT 文件缺失元数据，无法区分时间点。")
    print("   -> 必须启用 Plan B：去处理那个 .rds 文件。")

正在读取矩阵表头...
列名示例 (前5个): ['Noninjured_AAACCCAAGTGCTCAT "Noninjured_AAACCCAAGTTGTACC" "Noninjured_AAACCCACAGCGTGCT" "Noninjured_AAACCCACAGGTTCAT" "Noninjured_AAACCCAGTATCACGT" "Noninjured_AAACCCATCATGCGGC" "Noninjured_AAACCCATCGTTCATT" "Noninjured_AAACGAACACGGTGTC" "Noninjured_AAACGAAGTATGGAAT" "Noninjured_AAACGAATCTAGAGCT" "Noninjured_AAACGCTAGAGTCTTC" "Noninjured_AAACGCTAGCCTATCA" "Noninjured_AAACGCTGTGATGAAT" "Noninjured_AAACGCTGTGCCTATA" "Noninjured_AAACGCTTCTTCCCAG" "Noninjured_AAAGAACAGGAACTCG" "Noninjured_AAAGAACAGTTTGGCT" "Noninjured_AAAGAACCAGATCCAT" "Noninjured_AAAGAACGTAAGTTGA" "Noninjured_AAAGAACGTCCTGTTC" "Noninjured_AAAGAACTCGAACTCA" "Noninjured_AAAGGATCAATCCTTT" "Noninjured_AAAGGATCACTTGACA" "Noninjured_AAAGGATGTAGACAAT" "Noninjured_AAAGGATTCCGTGTCT" "Noninjured_AAAGGATTCGAACGGA" "Noninjured_AAAGGATTCGGCTTCT" "Noninjured_AAAGGATTCGGTTCAA" "Noninjured_AAAGGGCAGCCATTGT" "Noninjured_AAAGGGCAGGCTAACG" "Noninjured_AAAGGGCCACGCTGAC" "Noninjured_AAAGGGCCACTGGACC" "Noninjured_AAAG

In [ ]:
# @title 🏷️ 目录重构：赋予数据“语义化”名称
import os
import shutil

# 1. 设置基地路径
base_dir = '/content/drive/MyDrive/Project_Adipose_Ghost/00_RawData'

# 2. 定义重命名映射表 (Old Name -> New Name)
# 以后有新的要改，直接加在这个字典里
rename_map = {
    # [Old Path]             : [New Meaningful Path]
    'GSE138826'              : 'GSE138826_Muscle_CTX_TimeCourse',

    # 如果你之前还有没删干净的空文件夹，也可以列在这里
    # 'GSE189783'            : 'TRASH_Aging_Model'
}

print(f"🔧 启动目录重构工具，工作区: {base_dir}")

for old_name, new_name in rename_map.items():
    old_path = os.path.join(base_dir, old_name)
    new_path = os.path.join(base_dir, new_name)

    # 检查旧目录是否存在
    if os.path.exists(old_path):
        # 检查新目录是否已经被占用
        if not os.path.exists(new_path):
            try:
                os.rename(old_path, new_path)
                print(f"✅ 升级成功: {old_name} -> 📂 {new_name}")
            except OSError as e:
                print(f"❌ 重命名失败: {e}")
        else:
            print(f"⚠️ 跳过: 新目录 {new_name} 已存在。建议手动检查合并。")
    else:
        # 检查是否已经是新名字了
        if os.path.exists(new_path):
             print(f"👌 状态正常: 目录已经是最新名称 {new_name}")
        else:
             print(f"💨 目标未找到: {old_name} (可能还没下载?)")

print("\n✨ 目录结构优化完毕。")

🔧 启动目录重构工具，工作区: /content/drive/MyDrive/Project_Adipose_Ghost/00_RawData
✅ 升级成功: GSE138826 -> 📂 GSE138826_Muscle_CTX_TimeCourse

✨ 目录结构优化完毕。
